In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = '1-3m-linkedin-jobs-and-skills-2024:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4418374%2F7590738%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240605%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240605T055432Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3cbfa0f0aa38f2975101789af244cd885eeb13c81c11d915f3ae8f78c32d7892665fbd4046eb3e008bf15c543face85376635aed19ffe070b1cf68611029134df0822497ed93493797ef25cb201f839df949996a1f5f803fdd44e323512d70cdeb2e057c78c375998a94898c28c7ee2290bbe9c9033c44fbc20c1aae233aae4057e2e9a2ac4370f9526cd62e7aeb8e8dff9bf252f2be1cb95b8a94b8a5b5568eb7c3b99dc7d91c0791047d081280695d7d8870b1ac89380b7d3ec33f2b7fd4fc22cafce275f1f2f5490932169e4e2ffff270dc9316f18475ea2c7e5b78eba18d817d89e861759fbc3a88b69648e66573b17a9e86d12ca7cdd31823084b1f50f5'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# melihat jumlah skill

In [ ]:
import pandas as pd

file_path = "/kaggle/input/1-3m-linkedin-jobs-and-skills-2024/job_skills.csv"
df = pd.read_csv(file_path)
df.head(5)

In [ ]:
skills_column = df.iloc[:, 1]
skills = skills_column.str.split(',').explode()

In [ ]:
unique_skills = skills.str.strip().unique()
num_unique_skills = len(unique_skills)
print(f'jumlah skill yang tersedia: {num_unique_skills}')

In [ ]:
unique_skills_df = pd.DataFrame(unique_skills, columns=['Skill'])
unique_skills_df.head()

In [ ]:
print("Jumlah duplikasi: ", unique_skills_df.duplicated().sum())

In [ ]:
output_file_path = 'unique_skills.csv'
unique_skills_df.to_csv(output_file_path, index=False)